In [1]:
#cicla le giornate

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver import ActionChains
import time
import bs4
option = webdriver.ChromeOptions()
# I use the following options as my machine is a window subsystem linux. 
# I recommend to use the headless option at least, out of the 3
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-sh-usage')
# Replace YOUR-PATH-TO-CHROMEDRIVER with your chromedriver location
driver = webdriver.Chrome(r'C:\Users\matte\chromedriver',options=option)

URL=URL = "https://www.kickest.it/en/serie-a/stats/players/table/2021-2022?matchdays=1"
print(URL)
driver.get(URL)

#per prendere headers tabella, basta farlo una volta sola (controlla che non ci siano anomalie, ma non dovrebbero esserci)
array_headers=[]

headers=driver.find_elements(By.CLASS_NAME,"sf-statstable-thclick")
    
for header in headers:
        array_headers.append(header.text)
print(array_headers)
        
totalScrapedInfo=[]

for i in range(1, 38):
    URL="https://www.kickest.it/en/serie-a/stats/players/table/2021-2022?matchdays="+str(i)
    print(URL)
    driver.get(URL) # Getting page HTML through request
    print(driver)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(3)
    take_info_match(driver,soup,i)

https://www.kickest.it/en/serie-a/stats/players/table/2021-2022?matchdays=1


KeyboardInterrupt: 

In [18]:
#questo cicla le pagine riguardanti un' unica giornata
def take_info_match(driver,soup,matchday):
    for i in range(36):

        #qui dovresti chiamare una funzione che prende tutte le info come hai scritto sulle note sul tablet
        read_page(driver,soup,matchday)

        try:
                        button = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,'paginationjs-next.J-paginationjs-next')))
                        print ("Page is ready!")

                        print(button.text)
                        #driver.execute_script("arguments[0].click();",button);
                        #link=button.find_element(By.XPATH,"a")
                        
                        link=WebDriverWait(button, 8).until(EC.element_to_be_clickable((By.XPATH,'a')))
                        #link.click()
                        driver.execute_script("arguments[0].click();", link)
                        #Actions actions=new Actions(driver)

                        #time.sleep(8)
                        print(driver.title)
                        #element=driver.find_element(By.CLASS_NAME,"J4UujySvCMTOoHLer5tF.cdrkf9i9DtaKzSuOeUYx");
                        #print(element.text)

        except TimeoutException:
                        print ("Loading took too much time!")

        time.sleep(8) 

    page_source=driver.page_source
    soup=BeautifulSoup(page_source,'html.parser')

In [17]:
#qui si prendono le info di una pagina sola da 15 giocatori, riguardanti una giornata sola ovviamente
def read_page(driver,soup,matchday):
    # Make Python sleep for some time
    print(array_headers)
    time.sleep(2)

    # Obtain the number of rows in body
    rows = len(driver.find_elements_by_xpath(
        "/html/body/div[3]/div[5]/div/div[2]/table/tbody/tr"))

    # Obtain the number of columns in table
    cols = len(driver.find_elements_by_xpath(
        "/html/body/div[3]/div[5]/div/div[2]/table/tbody/tr[1]/td"))

    # Print rows and columns
    print(rows)
    print(cols)
    

    # Printing the data of the table
    for r in range(1, rows+1):
        scrapedInfo = {}
        statistiche={}
        statistiche_matchday={}
        for p in range(2, cols):
            
            # obtaining the text from each column of the table
            value = driver.find_element_by_xpath(
                "/html/body/div[3]/div[5]/div/div[2]/table/tbody/tr["+str(r)+"]/td["+str(p)+"]").text
            
            if p<5:
                if matchday==1:
                    scrapedInfo[array_headers[p-2]]=value
                else:
                    if p==2:
                        name_player=value
                        print(name_player)
            else:
                if(p>5):
                    statistiche_matchday[array_headers[p-2]]=value
        
        if matchday==1:
            statistiche["matchday"+str(matchday)]=statistiche_matchday
            scrapedInfo["statistiche"]=statistiche
        else:
            statistiche["matchday"+str(matchday)]=statistiche_matchday
            write_on_list(name_player,statistiche)
            #array_statistiche.append(statistiche)
        
        if matchday==1:
            print(scrapedInfo)
            totalScrapedInfo.append(scrapedInfo)  #totalScrapedInfo inizializzato all'inizio

In [16]:
def write_on_list(player,statistiche):
    for element in totalScrapedInfo:
        #print(element["Player"])
        if element["Player"]==player:
            element["statistiche"][list(statistiche.keys())[0]]=list(statistiche.values())[0]
            print(element)

In [20]:
import csv
import json

        
file = open('fantacalcio_provaFinal.json', mode='w', encoding='utf-8')
file.write(json.dumps(totalScrapedInfo))

1734856